In [1]:
import pandas as pd
import numpy as np
import utils.colonflag.feature_generation as fg
import matplotlib.pyplot as plt

In [2]:
dir = r'C:\Users\victo\OneDrive - University of Leeds\Documents\Uni Work\Project\MIMIC Work\Liver Cancer Prediction\liver_pred'
data_dir = dir + r"\data\interim"
data = pd.read_csv(data_dir + r"\processed_lab_data.csv", parse_dates=['charttime', 'index_date'])

In [3]:
binned_df = fg.bin_measurements(data)

In [42]:
binned_df['weekly_differences'] = round(binned_df['differences']/7, 0)
binned_df = binned_df[binned_df['differences']>-3]

In [4]:
pivot_df = binned_df.reset_index().pivot_table(index=['subject_id', 'weekly_differences'], columns='label', values='valuenum')

KeyError: 'weekly_differences'

In [ ]:
pivot_df

label                          Blood % Hemoglobin A1c  \
subject_id weekly_differences                           
10000980   0.0                                    NaN   
           6.0                                    NaN   
           15.0                                   NaN   
           19.0                                   NaN   
           20.0                                   NaN   
...                                               ...   
19999303   3.0                                    NaN   
           51.0                                   5.6   
           43.0                                   NaN   
           13.0                                   NaN   
           49.0                                   NaN   

label                          Blood Absolute Basophil Count  \
subject_id weekly_differences                                  
10000980   0.0                                          0.01   
           6.0                                           NaN   
           15.0                                          NaN   
           19.0                                          NaN   
           20.0                                          NaN   
...                                                      ...   
19999303   3.0                                           NaN   
           51.0                                          NaN   
           43.0                                          NaN   
           13.0                                          NaN   
           49.0                                          NaN   

label                          Blood Absolute Eosinophil Count  \
subject_id weekly_differences                                    
10000980   0.0                                            0.12   
           6.0                                             NaN   
           15.0                                            NaN   
           19.0                                            NaN   
           20.0                                            NaN   
...                                                        ...   
19999303   3.0                                             NaN   
           51.0                                            NaN   
           43.0                                            NaN   
           13.0                                            NaN   
           49.0                                            NaN   

label                          Blood Absolute Lymphocyte Count  \
subject_id weekly_differences                                    
10000980   0.0                                            0.77   
           6.0                                             NaN   
           15.0                                            NaN   
           19.0                                            NaN   
           20.0                                            NaN   
...                                                        ...   
19999303   3.0                                             NaN   
           51.0                                            NaN   
           43.0                                            NaN   
           13.0                                            NaN   
           49.0                                            NaN   

label                          Blood Absolute Monocyte Count  \
subject_id weekly_differences                                  
10000980   0.0                                          0.39   
           6.0                                           NaN   
           15.0                                          NaN   
           19.0                                          NaN   
           20.0                                          NaN   
...                                                      ...   
19999303   3.0                                           NaN   
           51.0                                          NaN   
           43.0                                          NaN   
   

In [ ]:
pivot_df.index.levels[1]

Index([  0.0,   6.0,  15.0,  19.0,  20.0,  21.0,  22.0,  25.0,  26.0,  27.0,
       ...
       583.0, 604.0, 612.0, 620.0, 625.0, 630.0, 634.0, 638.0, 568.0, 585.0],
      dtype='float64', name='weekly_differences', length=632)

In [ ]:
# Get all possible differences and blood test labels
all_differences = range(0, int(max(binned_df['weekly_differences']))+1)
all_blood_test_labels = pivot_df.columns.unique()

# Create a MultiIndex with all possible combinations of difference and subject_id
multiindex = pd.MultiIndex.from_product([pivot_df.index.levels[0], all_differences], names=['subject_id','differences'])

# Reindex the pivot table to ensure all combinations are present, filling missing values with NaN
pivot_df = pivot_df.reindex(multiindex)

In [ ]:
# 3d array with dimensions (subject_id, blood_test_label, difference)
array_3d = pivot_df.values.reshape((-1, len(pivot_df.columns), len(pivot_df.index.levels[1])))

In [ ]:
value = array_3d[0, :,:]

In [ ]:
array = fg.create_array_for_CNN(data, current_window_postindex=-3, max_history=150)

AttributeError: module 'utils.colonflag.feature_generation' has no attribute 'create_array_for_CNN'